In [38]:
from urllib.request import urlopen
from datetime import datetime, timedelta, timezone
import plotly.express as px
import pandas as pd
import json

In [14]:
(datetime.fromtimestamp(1636264800) - timedelta(hours=24)).timestamp()

1636178400.0

In [13]:
datetime.fromtimestamp(1636092000)

datetime.datetime(2021, 11, 5, 11, 0)

In [49]:
with open("owm_api_key", "r") as f:
    owm_api_key = f.readline().strip()

def get_pollution_data(lat, lon, data_type="forecast", start=0, end=0):
    if data_type == "forecast":
        url = f"http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={owm_api_key}"
    elif data_type == "current":
        url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={owm_api_key}"
    elif data_type == "history":
        url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}8&lon={lon}&start={start}&end={end}&appid={owm_api_key}"
    else:
        print("Unrecognized data type")
        return None
    
    result_string = urlopen(url).read().decode()
    
    result = json.loads(result_string)
    
    results_dict = {"datetime": [],
                   "co": [],
                   "no": [],
                   "no2": [],
                   "pm25": [],
                   "pm10": []}
    for item in result["list"]:
        components = item["components"]
        co = components["co"] / 1000
        no = components["no"] / 1000
        no2 = components["no2"] / 1000
        pm25 = components["pm2_5"] / 1000
        pm10 = components["pm10"] / 1000
        dt = datetime.fromtimestamp(item["dt"], tz=timezone(timedelta(hours=3), name="Europe/Moscow"))
        results_dict["datetime"].append(dt)
        results_dict["co"].append(co)
        results_dict["no"].append(no)
        results_dict["no2"].append(no2)
        results_dict["pm25"].append(pm25)
        results_dict["pm10"].append(pm10)
    
    dataframe = pd.DataFrame(results_dict)
    return dataframe

In [50]:
d1 = get_pollution_data(**stations[1])

In [52]:
d1

,datetime,co,no,no2,pm25,pm10
0,2021-11-07 09:00:00+03:00,0.25702,0.00000,0.00165,0.00050,0.00077
1,2021-11-07 10:00:00+03:00,0.25702,0.00003,0.00176,0.00050,0.00083
2,2021-11-07 11:00:00+03:00,0.25368,0.00007,0.00171,0.00050,0.00084
3,2021-11-07 12:00:00+03:00,0.25368,0.00009,0.00163,0.00050,0.00082
4,2021-11-07 13:00:00+03:00,0.25368,0.00005,0.00178,0.00050,0.00085
...,...,...,...,...,...,...
110,2021-11-11 23:00:00+03:00,0.27704,0.00000,0.00506,0.00332,0.00387
111,2021-11-12 00:00:00+03:00,0.27704,0.00000,0.00467,0.00345,0.00401
112,2021-11-12 01:00:00+03:00,0.27371,0.00000,0.00420,0.00369,0.00432
113,2021-11-12 02:00:00+03:00,0.27371,0.00000,0.00398,0.00410,0.00484


In [18]:
stations = {
        1: {"lat": 55.856324, "lon": 37.426628},
        2: {"lat": 55.833222, "lon": 37.525158},
        3: {"lat": 55.821154, "lon": 37.612592},
        4: {"lat": 55.811801, "lon": 37.71249},
        5: {"lat": 55.759354, "lon": 37.595584},
        6: {"lat": 55.715698, "lon": 37.6052377},
        7: {"lat": 55.658163, "lon": 37.471434},
        8: {"lat": 55.649412, "lon": 37.535874},
        9: {"lat": 55.635129, "lon": 37.658684},
        10: {"lat": 55.652695, "lon": 37.751502}
 
    }

In [23]:
current_results = []
for station in stations.values():
    result = get_pollution_data(station["lat"], station["lon"], "current")
    current_results.append(result)

In [33]:
for el1 in current_results:
    for el2 in current_results:
        if el1["list"] == el2["list"]:
            print("+", end=" ")
        else:
            print("-", end=" ")
    print()

+ + - - - - - - - - 
+ + - - - - - - - - 
- - + + - - - - - - 
- - + + - - - - - - 
- - - - + - + + - - 
- - - - - + - - + + 
- - - - + - + + - - 
- - - - + - + + - - 
- - - - - + - - + + 
- - - - - + - - + + 


In [32]:
current_results[0]

{'coord': {'lon': 37.4266, 'lat': 55.8563},
 'list': [{'main': {'aqi': 1},
   'components': {'co': 257.02,
    'no': 0.03,
    'no2': 1.76,
    'o3': 57.94,
    'so2': 0.52,
    'pm2_5': 0.5,
    'pm10': 0.83,
    'nh3': 0.06},
   'dt': 1636268400}]}

In [39]:
pd.to_datetime(datetime.fromtimestamp(1636268400, tz=timezone(timedelta(hours=3), name="Europe/Moscow")))

Timestamp('2021-11-07 10:00:00+0300', tz='Europe/Moscow')

In [60]:
d10_start = int(datetime(2021, 11, 5, 9, 0).timestamp())
d10_end = int(datetime.now().timestamp())
d10_history = get_pollution_data(stations[10]["lat"], stations[10]["lon"], "history", d10_start, d10_end)

In [61]:
d10_history

,datetime,co,no,no2,pm25,pm10
0,2021-11-05 07:00:00+03:00,0.37050,0.02213,0.03119,0.00884,0.01123
1,2021-11-05 08:00:00+03:00,0.38385,0.02682,0.03085,0.00834,0.01078
2,2021-11-05 09:00:00+03:00,0.37384,0.02503,0.03222,0.00746,0.00988
3,2021-11-05 10:00:00+03:00,0.35381,0.01877,0.03496,0.00703,0.00939
4,2021-11-05 11:00:00+03:00,0.34046,0.01196,0.03701,0.00714,0.00930
5,2021-11-05 12:00:00+03:00,0.33045,0.00520,0.03736,0.00664,0.00849
6,2021-11-05 13:00:00+03:00,0.32044,0.00169,0.03427,0.00574,0.00718
7,2021-11-05 14:00:00+03:00,0.31376,0.00094,0.03393,0.00537,0.00670
8,2021-11-05 15:00:00+03:00,0.31376,0.00060,0.03393,0.00593,0.00722
9,2021-11-05 16:00:00+03:00,0.32044,0.00043,0.03462,0.00723,0.00838


In [75]:
pd.to_datetime("2021-11-07 09:00").timestamp()

1636275600.0

In [76]:
datetime.fromtimestamp(1636275600)

datetime.datetime(2021, 11, 7, 14, 0)

In [62]:
from predictor import Predictor

In [63]:
P = Predictor()

In [64]:
d10_fact = P.get_data(10)

2021-11-06 23:35:40.598764
10_now expired
2021-11-06 22:54:01.372265
3600
3600


In [77]:
d10_fact.iat[-1, 0]

Timestamp('2021-11-08 09:00:00+0300', tz='Europe/Moscow')

In [68]:
d10 = d10_fact.merge(d10_history, on="datetime")

In [73]:
px.line(d10, x="datetime", y=["pm10_x", "pm10_y"])